In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
df_train = pd.read_csv('/kaggle/input/nlp-getting-started/train.csv')
df_train.head()

In [ ]:
df_train.info()

In [ ]:
df_train.target.value_counts()

In [ ]:
train = df_train.copy()

In [ ]:
train.drop(['keyword', 'location'], axis=1, inplace=True)

In [ ]:
import re
import nltk
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.tokenize import TweetTokenizer
from nltk.stem.wordnet import WordNetLemmatizer

nltk.download('wordnet')
nltk.download('stopwords')
nltk.download('punkt')
stop_words = nltk.corpus.stopwords.words(['english'])
print(stop_words)

lem = WordNetLemmatizer()

def cleaning(data):
  #remove urls
  tweet_without_url = re.sub(r'http\S+',' ', data)

  #remove hashtags
  tweet_without_hashtag = re.sub(r'#\w+', ' ', tweet_without_url)

  #3. Remove mentions and characters that not in the English alphabets
  tweet_without_mentions = re.sub(r'@\w+',' ', tweet_without_hashtag)
  precleaned_tweet = re.sub('[^A-Za-z]+', ' ', tweet_without_mentions)

    #2. Tokenize
  tweet_tokens = TweetTokenizer().tokenize(precleaned_tweet)
    
    #3. Remove Puncs
  tokens_without_punc = [w for w in tweet_tokens if w.isalpha()]
    
    #4. Removing Stopwords
  tokens_without_sw = [t for t in tokens_without_punc if t not in stop_words]
    
    #5. lemma
  text_cleaned = [lem.lemmatize(t) for t in tokens_without_sw]
    
    #6. Joining
  return " ".join(text_cleaned)

In [ ]:
train['cleaned_text'] = train['text'].apply(cleaning)
train.head()

In [ ]:
from textblob import TextBlob

def getSubjectivity(tweet):
  return TextBlob(tweet).sentiment.subjectivity

def getPolarity(tweet):
  return TextBlob(tweet).sentiment.polarity

In [ ]:
train['subjectivity'] = train['cleaned_text'].apply(getSubjectivity)
train['polarity'] = train['cleaned_text'].apply(getPolarity)
train.head()

In [ ]:
def getSentiment(score):
  if score < 0:
    return 'negative'
  elif score == 0:
    return 'neutral'
  else:
    return 'positive'

In [ ]:
train['sentiment'] = train['polarity'].apply(getSentiment)
train.head()

In [ ]:
test = pd.read_csv('/kaggle/input/nlp-getting-started/test.csv')
test.head()

In [ ]:
test.drop(['keyword', 'location'], axis=1, inplace=True)

In [ ]:
test['cleaned_text'] = test['text'].apply(cleaning)
test.head()

In [ ]:
test['subjectivity'] = test['cleaned_text'].apply(getSubjectivity)
test['polarity'] = test['cleaned_text'].apply(getPolarity)
test.head()

In [ ]:
test['sentiment'] = test['polarity'].apply(getSentiment)
test.head()

In [ ]:
encoding = {
    'negative':0,
    'neutral':1,
    'positive':2,
}
train.sentiment.replace(encoding, inplace=True)
test.sentiment.replace(encoding, inplace=True)

In [ ]:
print(train.head())
print(test.head())

In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import tensorflow.keras.layers as Layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from tensorflow.keras.optimizers import Adam
from keras.layers import Dense, Dropout, Embedding, LSTM, Conv1D, GlobalMaxPooling1D, Bidirectional, SpatialDropout1D
from keras.models import load_model

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

In [ ]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(train['cleaned_text'])

X_train = tokenizer.texts_to_sequences(train['cleaned_text'].values)
y_train = train[['sentiment']].values

X_test = tokenizer.texts_to_sequences(test['cleaned_text'].values)
y_test = test[['sentiment']].values

vocab_size = len(tokenizer.word_index)+1

In [ ]:
print("Vocabulary size: {}".format(vocab_size))
print("\n----------Example----------\n")
print("Sentence:\n{}".format(train["cleaned_text"][6]))
print("\nAfter tokenizing :\n{}".format(X_train[6]))

X_train = pad_sequences(X_train, padding='post')  # adding padding of zeros to obtain uniform length for all sequences
X_test = pad_sequences(X_test, padding='post')
print("\nAfter padding :\n{}".format(X_train[6]))

In [ ]:
EPOCHS = 100
BATCH_SIZE = 32
embedding_dim = 16
units = 256

model = Sequential()
model.add(
    Embedding(
        vocab_size, 
        embedding_dim, 
        input_length=X_train.shape[1]
        ))
model.add(SpatialDropout1D(0.4))
model.add(LSTM(units, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(3, activation='softmax')),
model.compile(loss='categorical_crossentropy', optimizer='adam',metrics=['accuracy'])
print(model.summary())

In [ ]:
Y = pd.get_dummies(train['sentiment']).values

print(X_train.shape, Y.shape)

Y_test = pd.get_dummies(test['sentiment']).values
print(X_test.shape, Y_test.shape)

In [ ]:
history = model.fit(
    X_train,
    Y, 
    epochs=10, 
    validation_split=0.25, 
#     validation_data=(X_test, y_test),
    batch_size=BATCH_SIZE)

In [ ]:
y_pred_test =  np.argmax(model.predict(X_test), axis=1)
print('Accuracy:\t{:0.1f}%'.format(accuracy_score(np.argmax(Y_test,axis=1),y_pred_test)*100))
print(classification_report(np.argmax(Y_test,axis=1), y_pred_test))